In [24]:
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import os
import time
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
import torch
from torch import nn, optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
import warnings
import shutil
from tqdm import tqdm # Import tqdm function directly
warnings.filterwarnings("ignore")

In [3]:
#Process image data for feature extraction using CNN
input_dir = '/content/CS610_AML_Group_Project/resized_images'
full_set = datasets.ImageFolder(input_dir)

In [141]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees = 15),
    transforms.ColorJitter(brightness=0.1, contrast = 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],std=[0.229,0.224,0.225]) #ImageNet
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
     transforms.Normalize([0.485,0.456,0.406],std=[0.229,0.224,0.225]) #ImageNet
])

train_size = int(0.7*len(full_set))
val_size = int(0.2*len(full_set))
test_size = len(full_set)-train_size-val_size
split_datasets = random_split(
    full_set,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

train_indices = split_datasets[0].indices
val_indices = split_datasets[1].indices
test_indices = split_datasets[2].indices

class CustomSubsetWithTransform(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform

    def __getitem__(self, idx):
        original_idx = self.indices[idx]
        img, label = self.dataset[original_idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.indices)
train_img_dataset = CustomSubsetWithTransform(full_set, train_indices, train_transform)
val_img_dataset = CustomSubsetWithTransform(full_set, val_indices, val_test_transform)
test_img_dataset = CustomSubsetWithTransform(full_set, test_indices, val_test_transform)

batch_size = 32
train_loader = DataLoader(train_img_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_img_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_img_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
print("Data processing done")

Data processing done


In [142]:
#get classes from directory
num_classes = len(full_set.classes)
class_names = full_set.classes
print("Number of classes (Full Set):", num_classes,"\n", full_set.classes)

Number of classes (Full Set): 50 
 ['adidas_forum_high', 'adidas_forum_low', 'adidas_gazelle', 'adidas_nmd_r1', 'adidas_samba', 'adidas_stan_smith', 'adidas_superstar', 'adidas_ultraboost', 'asics_gel-lyte_iii', 'converse_chuck_70_high', 'converse_chuck_70_low', 'converse_chuck_taylor_all-star_high', 'converse_chuck_taylor_all-star_low', 'converse_one_star', 'new_balance_327', 'new_balance_550', 'new_balance_574', 'new_balance_990', 'new_balance_992', 'nike_air_force_1_high', 'nike_air_force_1_low', 'nike_air_force_1_mid', 'nike_air_jordan_11', 'nike_air_jordan_1_high', 'nike_air_jordan_1_low', 'nike_air_jordan_3', 'nike_air_jordan_4', 'nike_air_max_1', 'nike_air_max_270', 'nike_air_max_90', 'nike_air_max_95', 'nike_air_max_97', 'nike_air_max_plus_(tn)', 'nike_air_vapormax_flyknit', 'nike_air_vapormax_plus', 'nike_blazer_mid_77', 'nike_cortez', 'nike_dunk_high', 'nike_dunk_low', 'puma_suede_classic', 'reebok_classic_leather', 'reebok_club_c_85', 'salomon_xt-6', 'vans_authentic', 'vans_

In [143]:
#check if cuda is available to use
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device, "is used")

cuda is used


In [152]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)

In [153]:
#Set up Loss and Optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [154]:
import torch
from sklearn.metrics import accuracy_score
from tqdm import tqdm # Assuming tqdm is imported if used in evaluation progress bar

# Ensure 'device' and 'criterion' are defined globally or passed as arguments
# Example:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# criterion = nn.CrossEntropyLoss() # Your loss function, e.g., CrossEntropyLoss

def evaluate(model, loader, epoch_name="Validation"):
    """
    Evaluates the model's performance on a given dataset loader.

    Args:
        model (torch.nn.Module): The neural network model to evaluate.
        loader (torch.utils.data.DataLoader): DataLoader for the dataset (validation or test set).
        epoch_name (str, optional): A name for the current evaluation phase (e.g., "Validation"). Defaults to "Validation".

    Returns:
        tuple: A tuple containing (average_loss, accuracy_score).
    """
    model.eval() # Set the model to evaluation mode (disables dropout, batchnorm updates, etc.)
    all_preds = []
    all_labels = []
    running_loss = 0.0

    with torch.no_grad(): # Disable gradient calculations to save memory and speed up computation
        loop = tqdm(loader, desc=f"{epoch_name} Epoch", leave=False) # Optional: progress bar for evaluation
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device) # Move data to the specified device (CPU/GPU)

            outputs = model(images) # Perform forward pass
            loss = criterion(outputs, labels) # Calculate the loss

            preds = torch.argmax(outputs, dim=1) # Get predicted class indices
            all_preds.extend(preds.cpu().numpy()) # Store predictions (move to CPU for numpy conversion)
            all_labels.extend(labels.cpu().numpy()) # Store true labels (move to CPU for numpy conversion)
            running_loss += loss.item() # Accumulate the loss for the current batch

    # Calculate overall accuracy and average loss
    acc = accuracy_score(all_labels, all_preds) if len(all_labels) > 0 else 0.0 # Calculate accuracy, handle empty labels
    avg_loss = running_loss / len(loader) if len(loader) > 0 else 0.0 # Calculate average loss, handle empty loader

    # Print evaluation results
    print(f"{epoch_name} Loss: {avg_loss:.4f}, Accuracy: {acc * 100:.2f}%")

    return avg_loss, acc # Return the average loss and accuracy

In [164]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

best_val_loss = float('inf')
patience = 5
epochs_no_improve = 0


def train(model, train_loader, val_loader, epochs=10, scheduler=None):
    global best_val_loss, patience, epochs_no_improve

    for epoch in range(epochs):

        model.train() #
        running_loss = 0.0
        all_preds = []
        all_labels = []
        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]", leave=False)

        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            running_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        train_acc = accuracy_score(all_labels, all_preds)
        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_acc * 100:.2f}%")

        val_loss, val_acc = evaluate(model, val_loader, epoch_name="Validation")
        if scheduler:
            if isinstance(scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Current Learning Rate: {current_lr:.6f}")


        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), '/content/CS610_AML_Group_Project/model_bank/best_cnn_resnet50.pth')
            print(f"Validation loss improved. Saving model to best_model.pth. Best Val Loss: {best_val_loss:.4f}")
        else:
            epochs_no_improve += 1
            print(f"Validation loss did not improve. Epochs without improvement: {epochs_no_improve}")

In [166]:
train(model, train_loader, val_loader, epochs=100)

Epoch 1/100 - Train Loss: 0.1484, Train Accuracy: 95.59%


Validation Loss: 0.8128, Accuracy: 79.71%
Validation loss did not improve. Epochs without improvement: 3


Epoch 2/100 - Train Loss: 0.1221, Train Accuracy: 96.54%


Validation Loss: 0.7586, Accuracy: 81.56%
Validation loss improved. Saving model to best_model.pth. Best Val Loss: 0.7586


Epoch 3/100 - Train Loss: 0.1063, Train Accuracy: 96.89%


...

Validation Loss: 0.9230, Accuracy: 82.25%
Validation loss did not improve. Epochs without improvement: 91


Epoch 99/100 - Train Loss: 0.0327, Train Accuracy: 98.90%


Validation Loss: 0.8836, Accuracy: 82.64%
Validation loss did not improve. Epochs without improvement: 92


Epoch 100/100 - Train Loss: 0.0284, Train Accuracy: 99.12%


Validation Loss: 0.9646, Accuracy: 80.63%
Validation loss did not improve. Epochs without improvement: 93
